In [1]:
# Prepare the notebook for use
import pandas as pd
pd.options.display.max_rows = 16
pd.options.display.precision = 3

# Database connection

First, you need to set database connection parameters if they are different from the default values.

In [2]:
from monroe_anal import set_connection_params

set_connection_params(host='localhost',
                      port=8086,
                      username='monroe',
                      password='secure',
                      database='monroe')

# Signal and node enumeration

MONROE project comprises hundreds of geospatially-scattered nodes that measure and report on various signals, each signal type stored in its own table. We can query available tables and available nodes with calls to `all_tables()` and `all_nodes()`, both of which return a list of items.

In [3]:
from monroe_anal import all_tables, all_nodes

all_tables()

['event', 'gps', 'modem', 'ping', 'sensor']

In [4]:
nodes = all_nodes()
# For brewity, show only first 10 nodes
nodes[:10]

['7', '23', '24', '26', '30', '39', '44', '53', '55', '57']

Since not all nodes necessarily appear in all tables (not all nodes are equipped with all sensors), we can use `nodes_for_table()` and `tables_for_node()` functions to query the nodes that appear in a particular table or tables that contain sensor measurements from a particular node. `nodes_for_table()` returns a mapping of table name to a list of nodes that are contained within, while `tables_for_node()` returns a set of tables.

In [5]:
from monroe_anal import tables_for_node, nodes_for_table

# For brewity, show the first six nodes of every table
{table: nodes[:7]
 for table, nodes in nodes_for_table().items()}

{'event': ['7', '23', '24', '26', '30', '39', '53'],
 'gps': ['39', '135', '144', '145', '237', '266', '308'],
 'modem': ['7', '23', '26', '30', '39', '53', '55'],
 'ping': ['7', '26', '39', '44', '53', '55', '57'],
 'sensor': ['7', '23', '24', '26', '30', '39', '53']}

Note, node 7 apparently doesn't have a GPS sensor.

In [6]:
tables_for_node('7')

{'event', 'modem', 'ping', 'sensor'}

# Fetching sensor data

The main user interface for fetching data is the `getdf()` function, which returns a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/10min.html) object with all queried signal tables merged, time sorted, resampled and interpolated, as required. This is its docstring:

In [7]:
from monroe_anal import getdf

help(getdf)

Help on function getdf in module monroe_anal.query_base:

getdf(tables, *, nodeid='', where='', limit=100000, start_time=None, end_time=None, freq=None, resample='', interpolate=False) -> pandas.core.frame.DataFrame
    Return MONROE data as Pandas DataFrame.
    
    Parameters
    ----------
    tables : str or list of str
        Table name(s) to query and merge. Tables can be from the list
        as retuend by ``all_tables()``.
    nodeid : int or str or list of int or str
        A single node ID or a list thereof. If empty, results for all
        available nodes are returned.
    where : str or list of str
        Additional SQL WHERE conditions.
    limit : int
        Hard-limit on the number of rows requested from the DB for each
        NodeId.
    start_time : str or datetime or pandas.Timestamp
        Query results after start time. Default is set to 14 days before
        `end_time` or the min timestamp of `tables`, whichever is later.
    end_time : str or datetime or 

For example, to fetch all data from tables _ping_ and _modem_ for the last 14 days, we could issue:

In [8]:
df = getdf('ping modem')
df

,Host,Operator,RTT,Iccid,NodeId,CID,DeviceMode,DeviceState,ECIO,Frequency,IP_Address,MCC_MNC,RSRQ,RSSI
time,,,,,,,,,,,,,,
2017-07-13 00:30:00,193.10.227.25,C-OTE,99.392,89300100150506375450,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-07-13 00:30:00,193.10.227.25,I WIND,NaN,8939883386101437693,308,36353550,3G,connected,NaN,2600,10.70.19.126,22288,-7.667,-74.000
2017-07-13 00:30:00,193.10.227.25,Telia N,88.278,8947080038000093176,358,33346582,3G,connected,NaN,800,10.248.145.13,24202,-9.489,-45.043
2017-07-13 00:30:00,193.10.227.25,I WIND,138.296,8939883386101437602,39,36370696,3G,connected,NaN,800,10.35.32.31,22288,-8.375,-73.000
2017-07-13 00:30:00,193.10.227.25,Telia N,80.101,8947080038000093184,454,33346572,3G,connected,NaN,800,10.248.185.112,24202,-8.489,-50.647
2017-07-13 00:30:00,193.10.227.25,I WIND,80.861,8939883386101437495,314,12090767,2G,connected,4.740,NaN,10.64.68.35,22288,NaN,-74.092
2017-07-13 00:30:00,193.10.227.25,Telia N,87.793,8947080038000093192,7,33346572,3G,connected,NaN,800,100.65.102.166,24202,-7.681,-35.217
2017-07-13 00:30:00,193.10.227.25,I WIND,119.243,8939883386101432348,322,36367366,3G,connected,NaN,800,10.34.187.37,22288,-9.473,-70.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Or, to fetch all available measurements of _gps_ and _ping_ data for node _39_ where the _Operator_ field contains an uppercase letter 'I' during the specified time, all resampled to 10-second intervals and linearly interpolated to avoid _NaN_ values:

In [9]:
df = getdf('ping gps', nodeid='39',
           start_time='2017-07',
           end_time='2017-08',
           where='Operator =~ /I/',
           freq='1s',
           resample='10s',
           interpolate=True)
df

,NodeId,Host,Operator,Iccid,Longitude,RTT,SatelliteCount,Latitude,Altitude,Speed
time,,,,,,,,,,
2017-07-13 00:00:00,39,193.10.227.25,voda IT,8939104160000392231,7.667,104.122,3.000,45.034,209.550,1.50
2017-07-13 00:00:00,39,193.10.227.25,I WIND,8939883386101437602,7.667,135.375,3.000,45.034,209.571,1.50
2017-07-13 00:00:10,39,193.10.227.25,voda IT,8939104160000392231,7.667,97.167,3.000,45.034,209.640,1.50
2017-07-13 00:00:10,39,193.10.227.25,I WIND,8939883386101437602,7.667,137.750,3.000,45.034,209.675,1.50
2017-07-13 00:00:20,39,193.10.227.25,voda IT,8939104160000392231,7.667,136.750,3.000,45.034,209.675,1.50
2017-07-13 00:00:20,39,193.10.227.25,I WIND,8939883386101437602,7.667,130.200,3.000,45.034,209.660,1.50
2017-07-13 00:00:30,39,193.10.227.25,voda IT,8939104160000392231,7.667,102.989,3.333,45.034,209.700,1.50
2017-07-13 00:00:30,39,193.10.227.25,I WIND,8939883386101437602,7.667,144.500,3.000,45.034,209.700,1.50
...,...,...,...,...,...,...,...,...,...,...


Note that for performance reasons, categorical fields are set to `pandas.Categorical` data types (encoding string values).

In [10]:
df.dtypes

NodeId            category
Host              category
Operator          category
Iccid             category
Longitude          float64
RTT                float64
SatelliteCount     float64
Latitude           float64
Altitude           float64
Speed              float64
dtype: object

Also notice how node 39 has three SIM card slots (connected to three operators), thus the resulting data frame contains data for three distinct [ICCIDs](https://en.wikipedia.org/wiki/Subscriber_identity_module).

In [11]:
df.Iccid.value_counts()

8939883386101437602     8640
8939104160000392231     8640
89390100001965068048    8583
Name: Iccid, dtype: int64

In [12]:
# Operator names contain capital I
df.Operator.value_counts()

I WIND     8640
voda IT    8640
TIM        8583
Name: Operator, dtype: int64

# Signal time range

The time range of signal reporting may vary per each signal type and per node. We can use the `table_timerange()` function to query the minimum and maximum measurement time in a table.

In [13]:
from monroe_anal import table_timerange

# Time range of all GPS measurements
min_time, max_time = table_timerange('gps')
min_time, max_time

(Timestamp('2017-07-13 00:00:00+0000', tz='UTC'),
 Timestamp('2017-07-13 23:59:59.100000+0000', tz='UTC'))

In [14]:
# Time range of event measurements corresponding to node '7'
table_timerange('event', nodeid='7')

(Timestamp('2017-07-13 00:00:12+0000', tz='UTC'),
 Timestamp('2017-07-13 23:50:19.070000+0000', tz='UTC'))

# Distinct field values

While we could use `getdf()` function to fetch the data before extracting unique values for a particular column, this may result in needlessly retrieving the whole table of data from the database. To avoid such an expensive call when we are only interested in unique (distinct) field values (e.g. all telecom operators the nodes are connected to when running the _ping_ experiment), we can use `distinct_values()` function. It accepts a table name and a field name of interest, along with (optionally) start and end date, node id, and where conditions (as always, see the docsting for more info).

In [15]:
from monroe_anal import distinct_values

distinct_values('ping', 'Operator')

{'242 14',
 '3',
 '3 SE',
 'Acc. Internet da cell',
 'C-OTE',
 'I TIM',
 'I WIND',
 'Movistar',
 'N Telenor',
 'NetCom',
 'Orange',
 'Orange Internet Móvil',
 'TELIA S',
 'TIM',
 'TIM WAP',
 'Telenor',
 'Telenor SE',
 'TelenorS',
 'Telia',
 'Telia N',
 'WIND WEB',
 'YOIGO',
 'Yoigo Internet',
 'voda ES',
 'voda IT',
 'vodafone IT'}

# Custom SQL queries

In advanced workflows, you can make custom SQL queries (as supported in [InfluxDB subset of SQL](https://docs.influxdata.com/influxdb/v1.3/query_language/data_exploration/)) using `query()` and `query_async()` functions. The former executes a query and returns a [`influxdb.resultset.ResultSet`](https://influxdb-python.readthedocs.io/en/latest/resultset.html) object while the latter yields these results objects in sequence as they become available.

In [16]:
from monroe_anal import query

result = query('SELECT COUNT(*) FROM ping_10ms')

list(result.get_points())

[{'count_Host': 12118024,
  'count_Operator': 12077653,
  'count_RTT': 10979433,
  'time': 0}]